In [1]:
!git clone https://github.com/shoaib6174/MultiWOZ-Clustering/


!pip install pyvis

Cloning into 'MultiWOZ-Clustering'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 105 (delta 54), reused 83 (delta 32), pack-reused 0
Receiving objects: 100% (105/105), 14.89 MiB | 5.81 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Checking out files: 100% (38/38), done.


In [2]:
import pandas as pd
import numpy as np

from pyvis.network import Network
from IPython.core.display import display, HTML

import collections
import math

In [3]:
def show_graph(df, min_weight=1):
    df_edges = df[["cluster_id", "next_utterance_cluster_id"]]
    df_edges = df_edges.dropna()
    df_edges.columns = ['Source', 'Target']
    df_edges_weights = df_edges.groupby(['Source','Target']).size().reset_index().rename(columns={0:'Weight'})
    df_edges_weights = df_edges_weights[df_edges_weights['Weight'] >= min_weight]
    print(len(df_edges_weights.index))
    dialogue_graph = Network(height='750px', width='100%', bgcolor='#222222', font_color='white', directed=True)
    dialogue_graph.barnes_hut()

    sources = df_edges_weights['Source']
    targets = df_edges_weights['Target']
    weights = df_edges_weights['Weight']

    edge_data = zip(sources, targets, weights)



    for edge in edge_data:
        src = edge[0]
        dst = edge[1]
        w = edge[2]

        dialogue_graph.add_node(src, src, title=src)
        dialogue_graph.add_node(dst, dst, title=dst)
        dialogue_graph.add_edge(src, dst, value=w)

    neighbor_map = dialogue_graph.get_adj_list()  #dictionary mapping of Node ID to list of Node IDs 

    # add neighbor data to node hover data (Just provides list of all connected nodes to a given node
    for node in dialogue_graph.nodes:
        node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
        node['value'] = len(neighbor_map[node['id']])

    dialogue_graph.show('dialogue_graph.html')

    display(HTML("dialogue_graph.html"))

# Import utterances df with clusters and nodes df

In [6]:
path = "/content/MultiWOZ-Clustering/ConvRT/dialogue_clusters_convRT_200_with_replacement.csv"
df = pd.read_csv(path)
df = df.set_index("utterance_id")
df

,Unnamed: 0,dialogue_id,services,services_list,utterance_no,next_utterance_id,speaker,text,cleaned_text,cluster_id,next_utterance_cluster_id
utterance_id,,,,,,,,,,,
0-SNG02022,0,SNG02022,hospital,['hospital'],0,1-SNG02022,USER,I am looking for the Addenbrookes Hospital wit...,i am looking for the addenbrookes hospital wit...,u-26,s-0
1-SNG02022,1,SNG02022,hospital,['hospital'],1,2-SNG02022,SYSTEM,The telephone number is 01223217712,the telephone number is 01223217712,s-0,u-8
2-SNG02022,2,SNG02022,hospital,['hospital'],2,3-SNG02022,USER,Thank you Goodbye,thank you goodbye,u-8,s-24
3-SNG02022,3,SNG02022,hospital,['hospital'],3,NaN,SYSTEM,Have a wonderful day!,have a wonderful day,s-24,NaN
0-PMUL4452,4,PMUL4452,restaurant & taxi & hotel,"['restaurant', 'taxi', 'hotel']",0,1-PMUL4452,USER,I am traveling to Cambridge and looking forwar...,i am traveling to cambridge and looking forwar...,u-7,s-0
...,...,...,...,...,...,...,...,...,...,...,...
15-MUL2568,33319,MUL2568,taxi & attraction & hotel,"['taxi', 'attraction', 'hotel']",15,16-MUL2568,SYSTEM,I have booked a car from the acorn to country ...,i have booked a car from the acorn to country ...,s-0,u-8
16-MUL2568,33320,MUL2568,taxi & attraction & hotel,"['taxi', 'attraction', 'hotel']",16,17-MUL2568,USER,No that was it. Thanks for your help!,no that was it thanks for your help,u-8,s-15
17-MUL2568,33321,MUL2568,taxi & attraction & hotel,"['taxi', 'attraction', 'hotel']",17,18-MUL2568,SYSTEM,So glad that I could assist. Please call again.,so glad that i could assist please call again,s-15,u-8


In [8]:
pd.set_option("display.max_colwidth", -1)
pd.set_option('display.max_rows', 1000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [9]:
path = "/content/MultiWOZ-Clustering/ConvRT/nodes_convRT_with_10_utterances.csv"
nodes_df = pd.read_csv(path)
nodes_df.columns = ["cluster_id", "10_utterances"]

nodes_df.head(10)

,cluster_id,10_utterances
0,u-0,"['i would like to book a table on thursday for 6 people at 1330', 'yes please i need a table on friday at 1230 for 3 people please', 'sorry this is the husband im taking over the call my wife is too indecisive please book us for nandos at 1345 on tuesday for 5 people thank you', 'book the one in the centre a table for 2 at 1345 on thursday', 'can i reserve a table for two at the moderately priced indian restaurant', 'yes can you make a reservation for 2 people for wednesday at 1200 please thanks', 'sounds good can you book me a table for 6 on monday at 1100', 'could you just book a table for me i would like it to be on monday at 1730 for 3 people please', 'that s it can you make a reservation for me there are 5 of us and we would like to reserve sunday at 1145 if possible', 'yes please make a reservation for 6 people at 1000']"
1,u-1,"['not really can you book one of them for friday for me', 'i do nt want to book at this time just the information so that s all thanks', 'yes i am ready for a reservation', 'wonderful you made me excited for this trip now ca nt wait that is everything thank you', 'actually i do nt need the room booked i think we re all set here have a good day', 'i ca nt wait to try it i m not ready to make a reservation yet thanks for your help i ll call back goodbye', 'no thank you please go ahead and make a reservation at meze bar', 'thank you so much no need for booking that is all of my questions', 'i only need to hotel information for now thank you for your help', 'actually i will book it myself allenbell sounds perfect thank you this is all i needed']"
2,u-2,"['just one what is the price to book the train', 'what is the cost per ticket', 'how much is admission', 'that is fine how much is the ticket', 'how much is that cinema s entry fee', 'i just want to know the price of the ticket', 'how much would the ticket cost', 'nevermind just remembered i may have to do something else that day could you just give me the price for a ticket that way i ll know and can get my own', 'i m not looking to book yet can you tell me their price ranges', 'what is price for a ticket and i also need the travel time']"
3,u-3,"['yeah can you get me a ticket', 'yes i ll need 4 tickets please', 'i will need 4 tickets please', 'i just need one ticket please', 'i will need two tickets', 'yes please i need 1 ticket', 'perfect i ll need 2 tickets', 'yes please book a ticket for one', 'yes please i ll actually need 7 tickets', 'yes please i need five tickets']"
4,u-4,"['just one please', 'yes please for 6 people', 'oh yeah five people please', 'it will only be me', 'just one', 'yes please for 4 people', 'yes for 3 people please', 'yes please for 6 people', 'yes for five please', '3 people']"
5,u-5,"['yes please for 2 nights', 'what about 3 nights', 'can you try booking it for just 1 night instead', 'just two nights please', 'try for one day', 'how about for 2 nights', 'how about for 2 nights', 'could you try four nights', 'no sorry i meant 2 nights', 'sure how about two nights']"
6,u-6,"['i just need it to be cheap and available on tuesday', 'ok thank you i would like the room for thursday', 'how about starting on friday', 'let s try for saturday but if that is not available then tuesday would be fine', 'that would be great please make it for monday', 'friday the same day please', 'thursday please', 'starting on monday please', 'how about thursday', 'it will be on friday']"
7,u-7,"['i m coming from norwich', 'i will be leaving from kings lynn', 'i want to leave from london liverpool street', 'yeseastside london', 'i will be departing from my house', 'i would like to go to norwich', 'i will be departing from 3 towns over', 'i am leaving from kohinoor', 'i need to go to kings lynn on thursday', 'i need to be dropped off at bridge guest house']"
8,u-8,"['that was all thanks', 'nothing more thank you', 'that is everything you were very helpful', 'thank you that s everything', 'that was everything thanks', 'th

#Graphs

### all Services

In [14]:
min_edge_weight = 1
show_graph(df, min_edge_weight)

478


### Service Wise Graph

In [15]:
set(df['services'])

{'attraction',
 'attraction & hotel',
 'attraction & train',
 'hospital',
 'hotel',
 nan,
 'restaurant',
 'restaurant & attraction',
 'restaurant & hotel',
 'restaurant & taxi & attraction',
 'restaurant & taxi & hotel',
 'restaurant & train',
 'taxi',
 'taxi & attraction & hotel',
 'train',
 'train & hotel'}

attraction

In [16]:
# write the name of services from the set above to check graph
services = ['attraction']


df_services = df[df["services"].isin(services)]
df_services

,Unnamed: 0,dialogue_id,services,services_list,utterance_no,next_utterance_id,speaker,text,cleaned_text,cluster_id,next_utterance_cluster_id
utterance_id,,,,,,,,,,,
0-SNG1058,766,SNG1058,attraction,['attraction'],0,1-SNG1058,USER,I'm looking for a place to go in town to have fun. Are there any nightclubs in the west?,i m looking for a place to go in town to have fun are there any nightclubs in the west,u-7,s-0
1-SNG1058,767,SNG1058,attraction,['attraction'],1,2-SNG1058,SYSTEM,"There are no nightclubs in the west, would another area work for you?",there are no nightclubs in the west would another area work for you,s-0,u-30
2-SNG1058,768,SNG1058,attraction,['attraction'],2,3-SNG1058,USER,How about a college in the west?,how about a college in the west,u-30,s-8
3-SNG1058,769,SNG1058,attraction,['attraction'],3,4-SNG1058,SYSTEM,"Sure, there are 5 colleges in the west part of town. How about Clare College?",sure there are 5 colleges in the west part of town how about clare college,s-8,u-30
4-SNG1058,770,SNG1058,attraction,['attraction'],4,5-SNG1058,USER,What's the address of that college?,what s the address of that college,u-30,s-0
5-SNG1058,771,SNG1058,attraction,['attraction'],5,6-SNG1058,SYSTEM,Clare College is located on Trinity Lane. Anything else?,clare college is located on trinity lane anything else,s-0,u-9
6-SNG1058,772,SNG1058,attraction,['attraction'],6,7-SNG1058,USER,"No, that would be it. Thanks.",no that would be it thanks,u-9,s-17
7-SNG1058,773,SNG1058,attraction,['attraction'],7,NaN,SYSTEM,You're quite welcome! Thanks for contacting Cambridge TownInfo Centre and have a great day!,you re quite welcome thanks for contacting cambridge towninfo centre and have a great day,s-17,NaN
0-PMUL1206,830,PMUL1206,attraction,['attraction'],0,1-PMUL1206,USER,I am looking for a college in the West in town.,i am looking for a college in the west in town,u-30,s-8


In [17]:
min_edge_weight = 1
show_graph(df_services, min_edge_weight)

84


# Repetation per cluster per dialogue

In [18]:
min_number_of_repetation_per_dialogue = 2
rep_per_cluster_per_dialogue = df.groupby(["dialogue_id", 'cluster_id'])["cluster_id"].count().to_frame().add_suffix('_Count').reset_index()
rep_per_cluster_per_dialogue = rep_per_cluster_per_dialogue[rep_per_cluster_per_dialogue["cluster_id_Count"] >= min_number_of_repetation_per_dialogue].reset_index().drop(['index'], axis=1)
rep_per_cluster_per_dialogue.columns = ["dialogue_id", "cluster_id", 'rep_count']
rep_per_cluster_per_dialogue

,dialogue_id,cluster_id,rep_count
0,MUL0002,s-0,6
1,MUL0002,u-7,6
2,MUL0005,s-0,8
3,MUL0005,u-7,7
4,MUL0007,s-0,5
...,...,...,...
5637,WOZ20644,u-7,2
5638,WOZ20654,s-0,2
5639,WOZ20654,u-7,2
5640,WOZ20673,s-0,4


counting rep_count

In [19]:
rep_per_cluster_per_dialogue.drop(['dialogue_id'], axis=1).groupby([ 'rep_count'])['cluster_id'].count().to_frame()

,cluster_id
rep_count,
2,1761
3,902
4,766
5,742
6,589
7,422
8,223
9,111
10,63


# Cluster repetition

### occurrence of cluster repitations sorted

In [20]:
min_Count_of_cluster_id = 1
occurrence_of_cluster_rep_per_cluster = rep_per_cluster_per_dialogue.groupby(['cluster_id'])['cluster_id'].count().to_frame().add_prefix('Count_of_').reset_index().sort_values(['Count_of_cluster_id'],ascending=False )
occurrence_of_cluster_rep_per_cluster.columns = ["cluster_id", 'num_of_repetation_occurances']
occurrence_of_cluster_rep_per_cluster = pd.merge(occurrence_of_cluster_rep_per_cluster, nodes_df, how='left')

occurrence_of_cluster_rep_per_cluster = occurrence_of_cluster_rep_per_cluster[occurrence_of_cluster_rep_per_cluster["num_of_repetation_occurances"] >= min_Count_of_cluster_id]
occurrence_of_cluster_rep_per_cluster.head(10)

,cluster_id,num_of_repetation_occurances,10_utterances
0,s-0,2323,"['what time would you like to arrive at castle galleries', 'absolutely do you have a particular arrival or departure time in mind today', 'when are you wanting to arrive', 'sure when would you like to arrive', 'no problem i just need to know what time you wish to get there and where you ll be arriving from', 'yes but i need the day that you will be arriving first', 'i can do that i just need to know what time you want to leave or arrive', 'what time would you like to arrive in cambridge', 'what time do you need to be picked up and from where', 'sure when would you like to arrive']"
1,u-7,2152,"['i m coming from norwich', 'i will be leaving from kings lynn', 'i want to leave from london liverpool street', 'yeseastside london', 'i will be departing from my house', 'i would like to go to norwich', 'i will be departing from 3 towns over', 'i am leaving from kohinoor', 'i need to go to kings lynn on thursday', 'i need to be dropped off at bridge guest house']"
2,u-0,473,"['i would like to book a table on thursday for 6 people at 1330', 'yes please i need a table on friday at 1230 for 3 people please', 'sorry this is the husband im taking over the call my wife is too indecisive please book us for nandos at 1345 on tuesday for 5 people thank you', 'book the one in the centre a table for 2 at 1345 on thursday', 'can i reserve a table for two at the moderately priced indian restaurant', 'yes can you make a reservation for 2 people for wednesday at 1200 please thanks', 'sounds good can you book me a table for 6 on monday at 1100', 'could you just book a table for me i would like it to be on monday at 1730 for 3 people please', 'that s it can you make a reservation for me there are 5 of us and we would like to reserve sunday at 1145 if possible', 'yes please make a reservation for 6 people at 1000']"
3,u-8,368,"['that was all thanks', 'nothing more thank you', 'that is everything you were very helpful', 'thank you that s everything', 'that was everything thanks', 'that should be everything thank you very much', 'that was everything thanks', 'that s everything actually thanks a ton bye', 'that was everything thank you', 'no that should be everything thank you very much']"
4,s-15,40,"['yes that is correct is there anything else i can help you with', 'it is free can i help you with anything else', 'do you need any further assistance', 'is there anything else i can help you with', 'i can help you with that are there any other requirements you are looking for', 'are you sure there is nothing else', 'do you need any further assistance', 'great is there anything else i can assist you with', 'is there anything else i can help you with', 'anything else i can look up for you']"
5,s-24,32,"['outstanding have a good rest of your day', 'great have a good day', 'great have an awesome day', 'wonderful have a great day', 'great hope you have a fantastic day', 'great have a great day', 'great have a good day', 'great have a nice day', 'great have a good day', 'great hope you have a good day']"
6,u-29,29,"['i will take the 521 can you please give me the travel time train id and price', 'what about the train id', 'yes can i have the train id and travel time please', 'let s go with 1324 please give me the arrival time and train id', 'yes one ticket i need the travel time and train id', 'i do nt need tickets just the information for the train i need the train id please', 'could you give me the train id too', 'yes i would like to book one seat that leaves on sunday please provide the price arrival time and train id thank you', 'yes that would work for mei will just need the price train id and travel time', 'it does nt matter i just need the travel time departure time and train id']"
7,s-8,26,"['there are several colleges in either the centre or west part of town which area do you prefer', 'there are 13 colleges in the center of town does entrance fee matter to you', 'there are no colleges in t

Count of how many times a cluster has repeated in a dialogue

In [21]:
repeated_clusters = rep_per_cluster_per_dialogue.groupby(['cluster_id', "rep_count"])['cluster_id'].count().to_frame().add_suffix('_Count').reset_index()
repeated_clusters.columns = ['cluster_id', 'rep_count_in_single_dialogue', 'count']

repeated_clusters.head(10)

,cluster_id,rep_count_in_single_dialogue,count
0,s-0,2,380
1,s-0,3,292
2,s-0,4,325
3,s-0,5,356
4,s-0,6,358
5,s-0,7,281
6,s-0,8,156
7,s-0,9,87
8,s-0,10,40
9,s-0,11,23


### sorted by  rep_count_in_single_dialogue and count

Read like: {cluster id} has repeated {rep_count_in_single_dialogue} times in {count} dialogues

In [22]:
repeated_clusters_sorted = repeated_clusters.sort_values(['rep_count_in_single_dialogue', 'count'],ascending=False )
repeated_clusters_sorted = pd.merge(repeated_clusters_sorted, nodes_df, how='left')
repeated_clusters_sorted.head(5)

,cluster_id,rep_count_in_single_dialogue,count,10_utterances
0,s-0,21,1,"['what time would you like to arrive at castle galleries', 'absolutely do you have a particular arrival or departure time in mind today', 'when are you wanting to arrive', 'sure when would you like to arrive', 'no problem i just need to know what time you wish to get there and where you ll be arriving from', 'yes but i need the day that you will be arriving first', 'i can do that i just need to know what time you want to leave or arrive', 'what time would you like to arrive in cambridge', 'what time do you need to be picked up and from where', 'sure when would you like to arrive']"
1,s-0,20,1,"['what time would you like to arrive at castle galleries', 'absolutely do you have a particular arrival or departure time in mind today', 'when are you wanting to arrive', 'sure when would you like to arrive', 'no problem i just need to know what time you wish to get there and where you ll be arriving from', 'yes but i need the day that you will be arriving first', 'i can do that i just need to know what time you want to leave or arrive', 'what time would you like to arrive in cambridge', 'what time do you need to be picked up and from where', 'sure when would you like to arrive']"
2,s-0,14,3,"['what time would you like to arrive at castle galleries', 'absolutely do you have a particular arrival or departure time in mind today', 'when are you wanting to arrive', 'sure when would you like to arrive', 'no problem i just need to know what time you wish to get there and where you ll be arriving from', 'yes but i need the day that you will be arriving first', 'i can do that i just need to know what time you want to leave or arrive', 'what time would you like to arrive in cambridge', 'what time do you need to be picked up and from where', 'sure when would you like to arrive']"
3,s-0,13,6,"['what time would you like to arrive at castle galleries', 'absolutely do you have a particular arrival or departure time in mind today', 'when are you wanting to arrive', 'sure when would you like to arrive', 'no problem i just need to know what time you wish to get there and where you ll be arriving from', 'yes but i need the day that you will be arriving first', 'i can do that i just need to know what time you want to leave or arrive', 'what time would you like to arrive in cambridge', 'what time do you need to be picked up and from where', 'sure when would you like to arrive']"
4,s-0,12,14,"['what time would you like to arrive at castle galleries', 'absolutely do you have a particular arrival or departure time in mind today', 'when are you wanting to arrive', 'sure when would you like to arrive', 'no problem i just need to know what time you wish to get there and where you ll be arriving from', 'yes but i need the day that you will be arriving first', 'i can do that i just need to know what time you want to leave or arrive', 'what time would you like to arrive in cambridge', 'what time do you need to be picked up and from where', 'sure when would you like to arrive']"


## dialogues sorted based on highest repetation

Read like: {dialogue_id} has {count} clusters which repeated {repetation_type} times

In [25]:
dialogue_rep = rep_per_cluster_per_dialogue.drop(['cluster_id'], axis=1).groupby([ 'dialogue_id','rep_count' ])['rep_count'].count().to_frame().add_suffix('_Count').reset_index()

dialogue_rep.columns = ["dialogue_id", "repetation_type", "count"]
dialogue_rep = dialogue_rep.sort_values(['repetation_type', 'count'],ascending=False ).reset_index()
dialogue_rep.drop(['index'],axis=1 , inplace=True)

dialogue_rep.head(50)

,dialogue_id,repetation_type,count
0,MUL1191,21,1
1,MUL1353,20,1
2,MUL0666,14,1
3,PMUL0413,14,1
4,PMUL3103,14,1
5,MUL0183,13,1
6,MUL1358,13,1
7,MUL1411,13,1
8,PMUL0043,13,1
9,PMUL2034,13,1


#### find dialogue with ids to check the utterances

In [26]:
dia_ids_to_find = [ "MUL1332" , 'MUL1191' ,'PMUL2587' ]

see repetation count of each cluster for the selected dialogues

In [27]:
rep_count_for_selected_dia = rep_per_cluster_per_dialogue[ rep_per_cluster_per_dialogue["dialogue_id"].isin(dia_ids_to_find)]
rep_count_for_selected_dia

,dialogue_id,cluster_id,rep_count
654,MUL1191,s-0,21
655,MUL1191,u-0,10
656,MUL1191,u-5,2
657,MUL1191,u-7,7
2769,PMUL2587,s-0,12
2770,PMUL2587,u-0,3
2771,PMUL2587,u-7,11


Check top 10 utterances of choosen cluster

In [28]:
cluster_ids = rep_count_for_selected_dia['cluster_id']
#cluster_ids = ["s-21", "s-23", "u-11", "u-41"]

nodes_df[nodes_df['cluster_id'].isin(cluster_ids)]

,cluster_id,10_utterances
0,u-0,"['i would like to book a table on thursday for 6 people at 1330', 'yes please i need a table on friday at 1230 for 3 people please', 'sorry this is the husband im taking over the call my wife is too indecisive please book us for nandos at 1345 on tuesday for 5 people thank you', 'book the one in the centre a table for 2 at 1345 on thursday', 'can i reserve a table for two at the moderately priced indian restaurant', 'yes can you make a reservation for 2 people for wednesday at 1200 please thanks', 'sounds good can you book me a table for 6 on monday at 1100', 'could you just book a table for me i would like it to be on monday at 1730 for 3 people please', 'that s it can you make a reservation for me there are 5 of us and we would like to reserve sunday at 1145 if possible', 'yes please make a reservation for 6 people at 1000']"
5,u-5,"['yes please for 2 nights', 'what about 3 nights', 'can you try booking it for just 1 night instead', 'just two nights please', 'try for one day', 'how about for 2 nights', 'how about for 2 nights', 'could you try four nights', 'no sorry i meant 2 nights', 'sure how about two nights']"
7,u-7,"['i m coming from norwich', 'i will be leaving from kings lynn', 'i want to leave from london liverpool street', 'yeseastside london', 'i will be departing from my house', 'i would like to go to norwich', 'i will be departing from 3 towns over', 'i am leaving from kohinoor', 'i need to go to kings lynn on thursday', 'i need to be dropped off at bridge guest house']"
32,s-0,"['what time would you like to arrive at castle galleries', 'absolutely do you have a particular arrival or departure time in mind today', 'when are you wanting to arrive', 'sure when would you like to arrive', 'no problem i just need to know what time you wish to get there and where you ll be arriving from', 'yes but i need the day that you will be arriving first', 'i can do that i just need to know what time you want to leave or arrive', 'what time would you like to arrive in cambridge', 'what time do you need to be picked up and from where', 'sure when would you like to arrive']"


check the uttereances of selected dialogues

In [29]:
df[df["dialogue_id"].isin(dia_ids_to_find)][["dialogue_id", 'services', "text", "cluster_id", "next_utterance_cluster_id"]]

,dialogue_id,services,text,cluster_id,next_utterance_cluster_id
utterance_id,,,,,
0-MUL1191,MUL1191,attraction & hotel,I am looking for a hotel with free wifi for when I am visiting Cambridge.,u-7,s-0
1-MUL1191,MUL1191,attraction & hotel,Acorn guest house in the north has a 4 star rating and free internet. Would you like me to book a room?,s-0,u-7
2-MUL1191,MUL1191,attraction & hotel,Could you tell me if Acorn guest house is in the moderate price range?,u-7,s-0
3-MUL1191,MUL1191,attraction & hotel,Yes it is in the moderate price range. Shall I book it for you?,s-0,u-0
4-MUL1191,MUL1191,attraction & hotel,"Yes, I need it for 4 people for 3 nights starting Thursday.",u-0,s-0
5-MUL1191,MUL1191,attraction & hotel,"I'm sorry, that is not available. Could you possibly shorten your stay? Also, this is a guesthouse, were you looking for a hotel room instead?",s-0,u-5
6-MUL1191,MUL1191,attraction & hotel,"Yeah, let's just try for one night please.",u-5,s-0
7-MUL1191,MUL1191,attraction & hotel,"Sorry, let me try to get this right. Are you looking for a moderate priced hotel(?) or guesthouse(?), with wifi, in the north? Does star number matter? For 1 night?",s-0,u-7
8-MUL1191,MUL1191,attraction & hotel,I want it to be a hotel and the stars don't matter.,u-7,s-0


# Cluster Flow in dialogue

flow of clusters in dialogues

In [31]:
df_flow = df.groupby(['dialogue_id'], as_index = False).agg({'cluster_id': ' '.join})
df_flow.head(20)

,dialogue_id,cluster_id
0,MUL0002,u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-8 s-24
1,MUL0005,u-7 s-0 u-0 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-8 s-21
2,MUL0007,u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-8 s-15 u-8 s-24
3,MUL0015,u-7 s-0 u-7 s-0 u-0 s-0 u-0 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-8 s-24
4,MUL0019,u-7 s-0 u-7 s-0 u-7 s-0 u-0 s-0 u-0 s-0 u-7 s-0 u-7 s-0 u-8 s-15 u-7 s-0 u-9 s-24
5,MUL0020,u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-0 s-0 u-5 s-0 u-7 s-0 u-8 s-18
6,MUL0022,u-7 s-0 u-0 s-0 u-20 s-0 u-7 s-0 u-7 s-0 u-5 s-0 u-7 s-3 u-7 s-0 u-7 s-3 u-7 s-0 u-8 s-24
7,MUL0023,u-7 s-0 u-7 s-0 u-0 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-16 s-0 u-0 s-0 u-0 s-0 u-0 s-12 u-0 s-0 u-8 s-24 u-8 s-27
8,MUL0024,u-7 s-0 u-7 s-0 u-0 s-0 u-5 s-0 u-7 s-0 u-7 s-0 u-0 s-0 u-7 s-0 u-7 s-10 u-9 s-15 u-8 s-24
9,MUL0030,u-7 s-0 u-0 s-0 u-16 s-0 u-7 s-0


Sorted count of repeating flows

In [32]:
flow_count = df_flow.groupby(['cluster_id'])['dialogue_id'].count().to_frame()
flow_count = flow_count.sort_values(['dialogue_id'], ascending=False).reset_index()

flow_count.columns = ["flow", "count"]
flow_count.head(10)

,flow,count
0,u-7 s-0 u-7 s-0 u-8 s-24,21
1,u-7 s-0 u-7 s-0 u-7 s-0 u-8 s-27,15
2,u-25 s-0 u-7 s-0 u-8 s-24,14
3,u-7 s-0 u-7 s-0 u-8 s-27,9
4,u-7 s-2 u-7 s-0 u-8 s-24,9
5,u-7 s-0 u-7 s-0 u-0 s-0 u-8 s-24,8
6,u-7 s-0 u-8 s-24,8
7,u-7 s-0 u-7 s-0 u-7 s-0,8
8,u-7 s-0 u-7 s-0 u-7 s-0 u-7 s-0 u-8 s-27,7
9,u-27 s-0 u-8 s-24,7


Getting Sub-flows from flows withn at least N turns

In [33]:
def allSubFlowsGreaterThanN(s,n):
    elements = s.split(" ")
    l = len(elements)
    if l < 5:
      return []
    elif l == 5:
      return [s]
    
    #print(elements)
    subs = []
    for i in range(0, l - n-1 ):
      for j in range(i+n, l+1):
        #print(i,j )
        subs.append("|".join(elements[i:j]))
    return subs



In [34]:
min_no_of_turns = 4
flows_substrings = []
for flow in flow_count["flow"]:
  flows_substrings += allSubFlowsGreaterThanN(flow, min_no_of_turns)

In [35]:
len(flows_substrings)

174886

Counting frequency of each sub-flows

In [36]:
frequency = collections.Counter(flows_substrings)
df_freq = pd.DataFrame(dict(frequency).items(), columns = ["sub-flow", "count"])
df_freq['length'] = df_freq.apply(lambda row: math.ceil(len(row['sub-flow'])/5), axis=1)
df_freq

,sub-flow,count,length
0,u-7|s-0|u-7|s-0,3466,3
1,u-7|s-0|u-7|s-0|u-8,477,4
2,u-7|s-0|u-7|s-0|u-8|s-24,158,5
3,u-7|s-0|u-7|s-0|u-7,1743,4
4,u-7|s-0|u-7|s-0|u-7|s-0,1508,5
...,...,...,...
88651,u-7|s-8|u-8|s-19|u-7|s-0|u-7|s-0,1,7
88652,s-8|u-8|s-19|u-7,1,4
88653,s-8|u-8|s-19|u-7|s-0,1,4
88654,s-8|u-8|s-19|u-7|s-0|u-7,1,5


Sorted frequency of subflows

In [37]:
df_freq_sorted = df_freq.sort_values(['count', "length"], ascending=False).reset_index()
df_freq_sorted = df_freq_sorted[df_freq_sorted['count'] > 1]
df_freq_sorted.head(10)

,index,sub-flow,count,length
0,0,u-7|s-0|u-7|s-0,3466,3
1,7,s-0|u-7|s-0|u-7,2441,3
2,8,s-0|u-7|s-0|u-7|s-0,2139,4
3,3,u-7|s-0|u-7|s-0|u-7,1743,4
4,4,u-7|s-0|u-7|s-0|u-7|s-0,1508,5
5,26,u-7|s-0|u-0|s-0,1100,3
6,33,s-0|u-7|s-0|u-7|s-0|u-7,1084,5
7,34,s-0|u-7|s-0|u-7|s-0|u-7|s-0,922,6
8,22,s-0|u-7|s-0|u-0,842,3
9,23,s-0|u-7|s-0|u-0|s-0,798,4
